In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from collections import defaultdict

from surprise.model_selection import KFold

from surprise.model_selection.validation  import cross_validate
from surprise.prediction_algorithms.knns import KNNWithMeans

import os
from surprise import Dataset
from surprise import Reader
from surprise import accuracy

from surprise.model_selection import train_test_split

from sklearn.metrics import  roc_curve
from sklearn.metrics import roc_auc_score
from collections import defaultdict

from surprise.prediction_algorithms.matrix_factorization import NMF
from surprise.prediction_algorithms.matrix_factorization import SVD

import seaborn as sns

import csv
import re


In [ ]:
bike_data = pd.read_csv('../input/day.csv')

bike_data


In [ ]:
video_data = pd.read_csv('../input/transcoding_mesurment.tsv', sep='\t')
video_data

In [ ]:
#question 1 heatmaps
#pandas dataframe.corr


bike_data_df = pd.DataFrame(data=bike_data)
video_data_df = pd.DataFrame(data=video_data)

bike_data_pearson = bike_data.corr(method='pearson', min_periods=1)
sns.heatmap(bike_data_pearson, center=0)

bike_pearson_abs = abs(bike_data_pearson)

#target variables - casual, registered, cnt

sorted_casual = bike_pearson_abs.casual.sort_values(ascending=False)
sorted_registered = bike_pearson_abs.registered.sort_values(ascending=False)
sorted_count = bike_pearson_abs.cnt.sort_values(ascending=False)


print("Bike Data List of absolute correlation with casual: \n" + str(sorted_casual))
print("Bike Data List of absolute correlation with registered: \n" + str(sorted_registered))
print("Bike Data List of absolute correlation with count: \n" + str(sorted_count))

In [ ]:
video_data_pearson = video_data.corr(method='pearson', min_periods=1)
video_pearson_abs = abs(video_data_pearson)
sns.heatmap(video_data_pearson, center=0)

#target variable is transcoding time "utime"

sorted_utime = video_pearson_abs.utime.sort_values(ascending=False)

print("Video Data List of absolute correlation with utime: \n" + str(sorted_utime))


In [ ]:
#question 2 plot histograms

plt.title("Casual user count")
plt.xticks(rotation='vertical')
plt.xlabel("Number")
plt.ylabel("Number of ")
plt.hist(bike_data.casual, bins='auto', align='left', rwidth=0.65)

plt.show()

plt.title("Registered user count")
plt.xticks(rotation='vertical')
plt.xlabel("Number")
plt.ylabel("Number of ")
plt.hist(bike_data.registered, bins='auto', align='left', rwidth=0.65)

plt.show()

plt.title("Count count")
plt.xticks(rotation='vertical')
plt.xlabel("Number")
plt.ylabel("Number of ")
plt.hist(bike_data.cnt, bins='auto', align='left', rwidth=0.65)

plt.show()

plt.title("Normalized Temperature count")
plt.xticks(rotation='vertical')
plt.xlabel("Number")
plt.ylabel("Number of ")
plt.hist(bike_data.temp, bins='auto', align='left', rwidth=0.65)

plt.show()

plt.title("Normalized Feeling Temperature count")
plt.xticks(rotation='vertical')
plt.xlabel("Number")
plt.ylabel("Number of ")
plt.hist(bike_data.atemp, bins='auto', align='left', rwidth=0.65)

plt.show()


plt.title("Windspeed count")
plt.xticks(rotation='vertical')
plt.xlabel("Number")
plt.ylabel("Number of ")
plt.hist(bike_data.windspeed, bins='auto', align='left', rwidth=0.65)

plt.show()

plt.title("Humidity count")
#Humidity "frequency?"
plt.xticks(rotation='vertical')
plt.xlabel("Number")
plt.ylabel("Number of ")
plt.hist(bike_data.hum, bins='auto', align='left', rwidth=0.65)

plt.show()


In [ ]:
#question 2 continued - video dataset

#everything except codec, height, width, o-height, o-width?

unique_duration = np.unique(video_data.duration)

plt.title("Duration count")
plt.xticks(rotation='vertical')
plt.xlabel("Number")
plt.ylabel("Number of ")

plt.hist(video_data.duration, bins=unique_duration, align='left', rwidth=0.65)

plt.show()


unique_bitrate = np.unique(video_data.bitrate)
append = np.ndarray.tolist(unique_bitrate)
append_to = append.append(max(append) + 1)

plt.title("Bitrate count")
plt.xticks(rotation='vertical')
plt.xlabel("Number")
plt.ylabel("Number of ")
plt.hist(video_data.bitrate, bins=append_to, align='left', rwidth=0.65)

plt.show()



plt.title("Framerate count")
plt.xticks(rotation='vertical')
plt.xlabel("Number")
plt.ylabel("Number of ")
plt.hist(video_data.framerate, bins='auto', align='left', rwidth=0.65)

plt.show()



plt.title("i Frames count")
plt.xticks(rotation='vertical')
plt.xlabel("Number")
plt.ylabel("Number of ")
plt.ylim([0,4000])
plt.hist(video_data.i, bins='auto', align='left', rwidth=0.65)

plt.show()

In [ ]:
#video data continued



plt.title("p Frames count")
plt.xticks(rotation='horizontal')
plt.xlabel("Number")
plt.ylabel("Number of ")
plt.ylim([0,1000])
plt.hist(video_data.p, bins='auto', align='left', rwidth=0.65)

plt.show()

plt.title("b Frames count")
plt.xticks(rotation='horizontal')
plt.xlabel("Number")
plt.ylabel("Number of ")
plt.hist(video_data.b, bins='auto', align='left', rwidth=0.65)

plt.show()

plt.title("Frames count")
plt.xticks(rotation='horizontal')
plt.xlabel("Number")
plt.ylabel("Number of ")
plt.ylim([0,2000])
plt.hist(video_data.frames, bins='auto', align='left', rwidth=0.65)

plt.show()

plt.title("i-size count")
plt.xticks(rotation='horizontal')
plt.xlabel("Number")
plt.ylabel("Number of ")
#plt.ylim([0,1000])
plt.hist(video_data.i_size, bins='auto', align='left', rwidth=0.65)

plt.show()


plt.title("p-size count")
plt.xticks(rotation='horizontal')
plt.xlabel("Number")
plt.ylabel("Number of ")
plt.ylim([0,6000])
plt.hist(video_data.p_size, bins='auto', align='left', rwidth=0.65)

plt.show()



plt.title("b-size count")
plt.xticks(rotation='horizontal')
plt.xlabel("Number")
plt.ylabel("Number of ")
plt.ylim([0,100000])
plt.hist(video_data.b_size, bins='auto', align='left', rwidth=0.65)

plt.show()



In [ ]:
# video data continued 

plt.title("size count")
plt.xticks(rotation='horizontal')
plt.xlabel("Number")
plt.ylabel("Number of ")
plt.ylim([0,5])
plt.hist(video_data.size, bins='auto', align='left', rwidth=0.65)

plt.show()

plt.title("o-bitrate count")
plt.xticks(rotation='horizontal')
plt.xlabel("Number")
plt.ylabel("Number of ")
plt.hist(video_data.o_bitrate, bins='auto', align='left', rwidth=0.65)

plt.show()

plt.title("o-framerate count")
plt.xticks(rotation='horizontal')
plt.xlabel("Number")
plt.ylabel("Number of ")
plt.hist(video_data.o_framerate, bins='auto', align='left', rwidth=0.65)

plt.show()

plt.title("o-height count")
plt.xticks(rotation='horizontal')
plt.xlabel("Number")
plt.ylabel("Number of ")
plt.hist(video_data.o_height, bins='auto', align='left', rwidth=0.65)

plt.show()


plt.title("o-width count")
plt.xticks(rotation='horizontal')
plt.xlabel("Number")
plt.ylabel("Number of ")
plt.hist(video_data.o_width, bins='auto', align='left', rwidth=0.65)

plt.show()



In [ ]:
unique_p = np.unique(video_data.p)
print(unique_p)
unique_b = np.unique(video_data.b)
print(unique_b)

In [ ]:
# question 3 box plots of categorical features vs target variable

#bike_data_df.boxplot(column=['casual', 'registered', 'cnt'], by='instant', figsize=(12,8))
#plt.show()

#bike_data_df.boxplot(column=['casual', 'registered', 'cnt'], by='dteday', figsize=(12,8))
#plt.show()

# casual is plotted separately because registered and cnt numbers are much higher than casual

#fig, ax = plt.subplots(figsize=(12,8))

bike_data_figsize = (10,6)
casual_figsize = (4,6)
bike_data_df.boxplot(column=['registered', 'cnt'], by='season', figsize=bike_data_figsize)
plt.show()

bike_data_df.boxplot(column=['casual'], by='season', figsize=casual_figsize)
plt.show()

bike_data_df.boxplot(column=['registered', 'cnt'], by='yr', figsize=bike_data_figsize)
plt.show()

bike_data_df.boxplot(column=['casual'], by='yr', figsize=casual_figsize)
plt.show()

bike_data_df.boxplot(column=['registered', 'cnt'], by='mnth', figsize=bike_data_figsize)
plt.show()

bike_data_df.boxplot(column=['casual'], by='mnth', figsize=casual_figsize)
plt.show()

bike_data_df.boxplot(column=['registered', 'cnt'], by='holiday', figsize=bike_data_figsize)
plt.show()

bike_data_df.boxplot(column=['casual'], by='holiday', figsize=casual_figsize)
plt.show()

bike_data_df.boxplot(column=['registered', 'cnt'], by='weekday', figsize=bike_data_figsize)
plt.show()

bike_data_df.boxplot(column=['casual'], by='weekday', figsize=casual_figsize)
plt.show()

bike_data_df.boxplot(column=['registered', 'cnt'], by='workingday', figsize=bike_data_figsize)
plt.show()

bike_data_df.boxplot(column=['casual'], by='workingday', figsize=casual_figsize)
plt.show()

bike_data_df.boxplot(column=['registered', 'cnt'], by='weathersit', figsize=bike_data_figsize)
plt.show()

bike_data_df.boxplot(column=['casual'], by='weathersit', figsize=casual_figsize)
plt.show()


#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.boxplot.html




In [ ]:
# question 3 continued - video data

video_data_figsize = (10,6)
#casual_figsize = (4,6)

video_data_df.boxplot(column=['umem'], by='codec', figsize=video_data_figsize)
plt.show()

video_data_df.boxplot(column=['utime'], by='codec', figsize=video_data_figsize)
plt.show()

#everything except codec, height, width, o-height, o-width?

video_data_df.boxplot(column=['umem'], by='height', figsize=video_data_figsize)
plt.show()

video_data_df.boxplot(column=['utime'], by='height', figsize=video_data_figsize)
plt.show()

video_data_df.boxplot(column=['umem'], by='width', figsize=video_data_figsize)
plt.show()

video_data_df.boxplot(column=['utime'], by='width', figsize=video_data_figsize)
plt.show()


video_data_df.boxplot(column=['umem'], by='o_height', figsize=video_data_figsize)
plt.show()

video_data_df.boxplot(column=['utime'], by='o_height', figsize=video_data_figsize)
plt.show()

video_data_df.boxplot(column=['umem'], by='o_width', figsize=video_data_figsize)
plt.show()

video_data_df.boxplot(column=['utime'], by='o_width', figsize=video_data_figsize)
plt.show()

In [ ]:
#question 4 bike sharing


#plot count number per day for a few months
month_range = pd.date_range('1/1/2011', periods=12)

jan_plot = []
feb_plot = []
mar_plot = []
apr_plot = []
may_plot = []
jun_plot = []
jul_plot = []

bike_date_jan = bike_data_df[['dteday','cnt']][:31]
bike_date_feb = bike_data_df[['dteday', 'cnt']][31:59]
bike_date_mar = bike_data_df[['dteday', 'cnt']][59:90]
bike_date_apr = bike_data_df[['dteday', 'cnt']][90:120]
bike_date_may = bike_data_df[['dteday', 'cnt']][120:151]
bike_date_jun = bike_data_df[['dteday', 'cnt']][151:181]
bike_date_jul = bike_data_df[['dteday', 'cnt']][181:212]

#print(bike_date_may)


for index, row in bike_date_jan.iterrows():
    #print(row['dteday'], row['cnt'])
    jan_plot.append(row['cnt'])

for index, row in bike_date_feb.iterrows():
    feb_plot.append(row['cnt'])

for index, row in bike_date_mar.iterrows():
    mar_plot.append(row['cnt'])

for index, row in bike_date_apr.iterrows():
    apr_plot.append(row['cnt'])
    
for index, row in bike_date_may.iterrows():
    may_plot.append(row['cnt'])

for index, row in bike_date_jun.iterrows():
    jun_plot.append(row['cnt'])

for index, row in bike_date_jul.iterrows():
    jul_plot.append(row['cnt'])


jan_axis = range(1,32,1)
feb_axis = range(1,29,1)
mar_axis = range(1,32,1)
apr_axis = range(1,31,1)
may_axis = range(1,32,1)
jun_axis = range(1,31,1)
jul_axis = range(1,32,1)

plt.plot(jan_axis, jan_plot)
plt.xlabel("Days")
plt.ylabel("Bike sharing Count")
plt.title("January 2011 Bike Sharing Count by Day")
plt.show()

plt.plot(feb_axis, feb_plot)
plt.show()

plt.plot(mar_axis, mar_plot)
plt.show()

plt.plot(apr_axis, apr_plot)
plt.show()

plt.plot(may_axis, may_plot)
plt.show()

plt.plot(jun_axis, jun_plot)
plt.show()

plt.plot(jul_axis, jul_plot)
plt.show()

In [ ]:
#question 5 utime plot, mean, median


utime_all = video_data_df['utime']

print(utime_all)

utime_axis = range(0,len(video_data_df.index))
plt.plot(utime_axis, utime_all)

print("Mean transcoding time: " +str(np.mean(utime_all)))

print("Median transcoding time: " +str(np.median(utime_all)))


In [ ]:
#question 6 - theoretical

In [ ]:
# question 7 standardization

#center the data to have 0 mean and unit variance


#example below from
#https://www.analyticsvidhya.com/blog/2020/04/feature-scaling-machine-learning-normalization-standardization/
# data normalization with sklearn
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(X_train)

# transform training data
X_train_norm = norm.transform(X_train)

# transform testing dataabs
X_test_norm = norm.transform(X_test)


#########################################################
# data standardization with  sklearn
from sklearn.preprocessing import StandardScaler

# copy of datasets
X_train_stand = X_train.copy()
X_test_stand = X_test.copy()

# numerical features
num_cols = ['Item_Weight','Item_Visibility','Item_MRP','Outlet_Establishment_Year']

# apply standardization on numerical features
for i in num_cols:
    
    # fit on training data column
    scale = StandardScaler().fit(X_train_stand[[i]])
    
    # transform the training data column
    X_train_stand[i] = scale.transform(X_train_stand[[i]])
    
    # transform the testing data column
    X_test_stand[i] = scale.transform(X_test_stand[[i]])

In [ ]:
#running comment